### Execution time comparison 
#### Used for Table 3

### Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib import animation
from IPython.display import HTML
plt.rcParams['text.usetex'] = True
from piston_Lagrangian import setup

### Auxiliary functions

In [ ]:
def generate_input_CLAW(method):
    far_field_damping = False #Exponential far_field_damping (w. source term)
    scalar_far_field =False
    relaxation_method = False
    matrix_filter=False
    integral_source = False
    euler_RS = 'euler'
    solver_class = 'sharpclaw_custom'
    far_field_damping_rate = 30.0
    damping_function = "Mayer"
    slowing_function = "Mayer"


    if method == "RM":
        #Sponge layer-based absorbing boundary conditions
        #Using scalar weight function
        relaxation_method = True #Sponge layer far_field_damping

    elif method == "RM-SO":
        #Sponge layer-based absorbing boundary conditions
        #Using scalar far_field_damping function + Slowing down operator
        euler_RS = 'euler_slowing'
        relaxation_method = True #Sponge layer far_field_damping

    elif method == "RM-M":
        #Sponge layer-based absorbing boundary conditions
        #Using matrix far_field_damping function
        euler_RS = 'euler'
        relaxation_method = True
        matrix_filter = True

    elif method == "RM-M-SO":
        #Sponge layer-based absorbing boundary conditions
        #Using matrix far_field_damping function + Slowing down operator
        euler_RS = 'euler_slowing'
        relaxation_method = True
        matrix_filter = True
    
    elif method == "SDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler_slowing'
        far_field_damping = True #Exponential far_field_damping (w. source term)
        solver_class = 'sharpclaw'
        slowing_function = "Mayer"
        damping_function = "Engsig-Karup2"

    elif method == "S-SDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler_slowing'
        far_field_damping = True #Exponential far_field_damping (w. source term)
        scalar_far_field =True
        solver_class = 'sharpclaw'
        slowing_function = "Mayer"
        damping_function = "Engsig-Karup2"
    
    elif method == "NDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler'
        far_field_damping = True #Exponential far_field_damping (w. source term)
        integral_source = True
        solver_class = 'sharpclaw'
        far_field_damping_rate = 20.0

    elif method == "SNDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler_slowing'
        far_field_damping = True #Exponential far_field_damping (w. source term)
        integral_source = True
        solver_class = 'sharpclaw'

    
    return (euler_RS, relaxation_method, far_field_damping, matrix_filter,
                scalar_far_field, integral_source, solver_class, far_field_damping_rate,
                damping_function, slowing_function)

In [ ]:
def set_and_run_CLAW(method, data):
    (tfinal,xmax2,mx2,piston_freq,M,
    start_slowing,stop_slowing,
    start_absorbing,stop_absorbing) = data
    (euler_RS, relaxation_method, far_field_damping,
    matrix_filter, scalar_far_field,
    integral_source, solver_class,far_field_damping_rate,
    damping_function, slowing_function) = generate_input_CLAW(method)
    piston_problem2 = setup(tfinal=tfinal, xmax=xmax2, mx=mx2, M=M, CFL=0.8,limiting=1,nout=100,
                        solver_type=solver_class, order=2, time_integrator='Heun',
                        euler_RS=euler_RS, 
                        relaxation_method=relaxation_method,
                        far_field_damping=far_field_damping,
                        far_field_damping_rate=far_field_damping_rate,
                        matrix_filter=matrix_filter,
                        start_slowing=start_slowing, stop_slowing=stop_slowing,
                        start_absorbing=start_absorbing, stop_absorbing=stop_absorbing,
                        piston_freq=piston_freq,
                        sigma_damping_type=damping_function, 
                        sigma_slowing_type=slowing_function,
                        scalar_far_field=scalar_far_field,
                        integral_source=integral_source) #Not really using implicit integrator at the moment
    piston_problem2.verbosity = 0
    piston_problem2.run()
    piston_problem2.solver.__del__()
    return piston_problem2

### Problem setup

In [ ]:
start_slowing = 20*np.pi
start_absorbing = 20*np.pi

stop_slowing = 40*np.pi #10 periods in the sponge layer
stop_absorbing = 40*np.pi #10 periods in the sponge layer

p=20
p_domain=30
M=0.4
verbose=True
N_list = [10,50,250]
piston_freq = 1.0

#Reference solution
xmax2 = p_domain*2*np.pi
tfinal = p*2*np.pi



## Timing comparison

In [ ]:
import timeit
method_list = ["RM","RM-M","S-SDO","SDO", "NDO"]
time_list = {str(N):[] for N in N_list}

In [ ]:
time_list

In [ ]:
for N in N_list:
    mx2 = p_domain*N #Number of grid points
    data = (tfinal,xmax2,mx2,piston_freq,M,
            start_slowing,stop_slowing,
            start_absorbing,stop_absorbing) 
    for method in method_list:
        time = %timeit -o set_and_run_CLAW(method, data)
        time_list[str(N)].append(time.best)

In [ ]:
#Save dictionary
import pickle
with open('Timing_results.pkl', 'wb') as f:
    pickle.dump(time_list, f)

In [ ]:
#Load dictionary
with open('Timing_results.pkl', 'rb') as f:
    time_list = pickle.load(f)

In [ ]:
#Print dictionary as table
from tabulate import tabulate
table = []
for N in N_list:
    row = [N]
    for method in method_list:
        row.append(time_list[str(N)][method_list.index(method)])
    table.append(row)
print(tabulate(table, headers=["N"]+method_list, tablefmt='orgtbl'))
